# Case Study: US Drugs Database on AWS RDS

## MySQL drugsdb
### Author: Dr. Esma Yildirim

In this case study, we are going to create a `mysql` database instance using `RDS` service and another instance to connect to it using `EC2` service. 

Additional materials: 
- drugs_product.csv
- drugs_package.csv

If you already have an RDS and an EC2 instance up and running, go to Step 4.

### Step1: Create two security groups.
- rds-security-group: will be assigned to RDS instance. This security group must have an inbound rule allowing MySQL/Aurora connection on port 3306. It should be set so that the security group of the EC2 instance (rds-ec2-security) can connect to it to ensure security. 

- rds-ec2-security-group: will be assigned to EC2 instance. This security group must have two inbound rules. The first one must allow SSH connection on port 22 from MyIP only. The second rule must allow MySQL/Aurora connection on port 3306 to only rds-security-group. 

This way, we can connect to EC2 instance from outside and the EC2 instance can connect to RDS instance only. 

### Step2: Create RDS database. 
- Create a database with database instance id, username and password. If you want to create a database, it can be done if you give a database name. 
- The VPC, subnet and availability zones must the the same as EC2 instance. 
- The database name: drugsdb
- Assign rds-security-group as the security group of the instance. 

### Step3: Create EC2 instance.
- Use the default parameters to create an EC2 instance. 
- Assign rds-ec2-security-group as the security group of the instance. 

### Step4. Connect to EC2 instance using SSH with your pem file. Then use the following command to connect to RDS instance from your EC2 instance:

```
mysql -u admin -p -h mydbhost.cy86rwnhmv8p.us-east-1.rds.amazonaws.com
```
Here admin is the username, password will be entered after the command is executed. hostname is available on RDS console after the database is created. 

If you have not created a database named `drugsdb`, use the following sql command

```
create database drugsdb

```
### Step5. Create the database tables 

Analyze the csv files with Excel and draw an Entity-Relationship Diagram showing Primary Key and Foreign Key fields. 
Take a look at the columns at the CSV file and decide on the type of the columns: TEXT ? VARHAR(255) ? DATE ?
Which columns can be NOT NULL?

Make sure that the table field names match the order and the column names in the csv file when you write your create table command. 

```
USE drugsdb
DROP TABLE IF EXISTS drugs_product;
DROP TABLE IF EXISTS drugs_package;
```


```
CREATE TABLE drugs_product (
 
 FILL IN THIS AREA
 
);

CREATE TABLE drugs_package (


  FILL IN THIS AREA



);

```

### Step6. Upload the csv files into your EC2 instance using `scp` and run the following commands to insert entries into your tables

For Mac and Linux users

```scp -i <key file path> <local file path of csv file> ec2-user@<ip address>:/home/ec2-user/```

For Windows users

```pscp.exe -i <key file path> <local file path of csv file> ec2-user@<ip address>:/home/ec2-user/```

```
LOAD DATA LOCAL INFILE 'drugs_product.csv' INTO TABLE drugs_product CHARACTER SET latin1 FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"' IGNORE 1 lines;

load data local infile 'drugs_package.csv' into table drugs_package fields terminated by ',' enclosed by '\"' lines terminated by "\r\n" ignore 1 lines;

```


### Step7. Run the following Python code on EC2 instance to execute some SQL queries. Make sure you install pymysql, pandas and matplotlib modules with the following:

```
pip install pandas
pip install pymysql
pip install matplotlib
```

In [ ]:
# Selecting Data Subsets
# AWS RDS Mysql example
# Author: Dr. Esma Yildirim
# Kaggle US Health Informatics Dataset
# Drugs database
import pymysql
import pandas as pd
import matplotlib.pyplot as plot

connection = pymysql.connect(host= 'drugsdbinstance.cy86rwnhmv8p.us-east-1.rds.amazonaws.com',
                             user= 'admin', password='', database='drugsdb')

pd.options.display.max_columns = 100

# 1. List all distinct product proprietyname and routname fields of which routename is 'topical'
df = pd.read_sql("""SELECT DISTINCT proprietaryname, routename FROM drugs_product where LOWER(routename) = 'topical'""", connection)
print(df)

#EXERCISE 2. List all  proprietaryname,productid and startmarketingdate fields where starting marketing date is after 2005 January first



#EXERCISE 3. List all distinct proprietary names along with ndccodes where name starts with an 'O'



#EXERCISE 4. List all proprietaryname,dosageformnames,labelernames and startmarketingdate  for the drug 'Zprexa' ordered by their startmarketingdate descending order and labelername in ascending order.



#EXERCISE 5. List productndc,proprietartname and labelername of all tablet drugs from the company GlaxoSmithKline ...


# 6. How many different route names  are there for the product named 'humulin'?

df = pd.read_sql("""SELECT count(distinct routename) from drugs_product WHERE LOWER(proprietaryname) = 'humulin'""",
             connection)
print()
print(df)
            
# 7. How many distinct drugs labelers produce? Which one produces largest number of products?
df = pd.read_sql("""SELECT count(DISTINCT proprietaryname) AS cnt, labelername from drugs_product group by labelername order by cnt""",
             connection)
print()
print(df)

# 8. List product name, packagendccode product ndc code and package description for 'Xanax'
df = pd.read_sql("""SELECT T2.proprietaryname, T1.ndcpackagecode, T1.productndc, T1.packagedescription 
                    from drugs_package T1 
                     inner join drugs_product T2 on T1.productid = T2.productid 
                     where lower(proprietaryname) = 'xanax'""",
             connection)
print()
print(df)

#EXERCISE 9. List proprietary name and package description where package description includes the word 'syringe'




# 10. List the labelernames who produces more than 50 unique products order them from min to max.
df = pd.read_sql("""SELECT count(distinct proprietaryname) as cnt, labelername from drugs_product group by labelername HAVING cnt > 50 ORDER BY cnt ASC""",
             connection)
print()
print(df)

#EXERCISE 11. List count the number of distinct product proprietaryname from drugs_product table grouped by routename and ordered by the count from max to min




# GRAPHICS - LINE AND BAR GRAPHS
# 12. Create a bar graph to display how many distinct products Uriel Pharmacy starts marketing each year
df = pd.read_sql("""SELECT count(distinct proprietaryname) as product_count, substring(startmarketingdate, 1, 4) 
                    as year from drugs_product where labelername like 'Uriel%' group by year""",
             connection)
print()
print(df)
plot = df.plot.bar(x="year",y = "product_count", rot=70, title="Uriel Pharmacy - Product by Year")
plot.get_figure().savefig("Query10.png")

#EXERCISE 13. Create a year by year line graph for number of products marketed by 'Ventura Corporation'






# Closing the Database
connection.close()

